In [ ]:
import os
import pandas as pd

DPI = 300
FILE_FORMAT = 'jpg'
SAVE = False

def get_name_gen(max: int = 100):
    dir_name = os.path.join('images', 'experiment_logs_exploration')
    os.makedirs(dir_name, exist_ok=True)
    for i in range(1, max):
        yield os.path.join(dir_name, '{}.{}'.format(i, FILE_FORMAT))
name_gen = get_name_gen()

EXPERIMENT_ID = 1
metrics_df = pd.read_csv(os.path.join('..', 'logs', 'experiment', 'version_{}'.format(EXPERIMENT_ID), 'metrics.csv'))

In [ ]:
metrics_df.head()

In [ ]:
metrics_df.info()

In [ ]:
metrics_df.describe()

In [ ]:
num_epochs = int(metrics_df['epoch'].dropna().max()) + 1
num_epochs

In [ ]:
val_coco_df = metrics_df[['coco_stat_{}'.format(i) for i in range(12)]].dropna()
val_coco_df, test_coco_df = val_coco_df[:num_epochs], val_coco_df[num_epochs:]

In [ ]:
val_coco_df

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(num_epochs), val_coco_df['coco_stat_0'], 'o--')
plt.ylabel('Dokładność (%)')
plt.xlabel('Epoki')
plt.title('AP IoU=.50:.05:.95 na zbiorze walidacyjnym po każdej epoce - eksperyment {}'.format(EXPERIMENT_ID))
plt.grid()
plt.tight_layout()
if SAVE:
    plt.savefig(next(name_gen), dpi=DPI)
plt.show()

In [ ]:
test_coco_df

In [ ]:
x = range(len(metrics_df['loss'].dropna()))
window_size = int(.005 * len(metrics_df['loss'].dropna()))
plt.plot(x, metrics_df['loss'].dropna(), label='strata')
plt.plot(x, metrics_df['loss'].dropna().rolling(window=window_size).mean(), label='ruchoma średnia straty (okno={})'.format(window_size))#'rolling mean loss (window size={})'
plt.plot(x, metrics_df['loss'].dropna().rolling(window=window_size).std(), label='ruchoma średnia odchylenia standardowego (okno={})'.format(window_size))#'rolling loss std (window size={})'
plt.legend()
plt.ylabel('Wartość straty')
plt.xlabel('Iteracje')
plt.title('Wartość straty podczas uczenia - eksperyment {}'.format(EXPERIMENT_ID))
plt.grid()
plt.tight_layout()
if SAVE:
    plt.savefig(next(name_gen), dpi=DPI)
plt.show()

In [ ]:
metrics_df['loss'].dropna().describe()

In [ ]:
losses = {
    'ruchoma średnia straty klasyfikatora (okno={})'.format(window_size): metrics_df['loss_classifier'].dropna().rolling(window=window_size).mean(),
    'ruchoma średnia straty regresji pola ograniczającego (okno={})'.format(window_size): metrics_df['loss_box_reg'].dropna().rolling(window=window_size).mean(),
    'ruchoma średnia straty zawierania obiektu (okno={})'.format(window_size): metrics_df['loss_objectness'].dropna().rolling(window=window_size).mean(),
    'ruchoma średnia straty regresji pola ograniczającego (rpn) (okno={})'.format(window_size): metrics_df['loss_rpn_box_reg'].dropna().rolling(window=window_size).mean()
}
plt.stackplot(x, losses.values(), labels=losses.keys())
plt.legend(bbox_to_anchor=(1, .5), loc='center left')
plt.ylabel('Wartość straty')
plt.xlabel('Iteracje')
plt.title('Skład średniej straty podczas uczenia - eksperyment {}'.format(EXPERIMENT_ID))
plt.grid()
if SAVE:
    plt.savefig(next(name_gen), dpi=DPI, bbox_inches='tight')
plt.show()

In [ ]:
(metrics_df['step_end'] - metrics_df['step_start']).mean()

In [ ]:
gpu_memory = metrics_df['gpu_id: 0/memory.used (MB)'].dropna()
plt.plot(range(len(gpu_memory)), gpu_memory)
plt.title('Wykorzystanie pamięci GPU - eksperyment {}'.format(EXPERIMENT_ID))
plt.xlabel('Iteracje')
plt.ylabel('Wykorzystanie pamięci (MB)')
plt.grid()
plt.tight_layout()
if SAVE:
    plt.savefig(next(name_gen), dpi=DPI)
plt.show()